## Loading the Inception model from the [Applications of Keras](https://keras.io/applications/) or [Transfer learning with a pretrained ConvNet](https://www.tensorflow.org/tutorials/images/transfer_learning)
Keras Applications are deep learning models that are made available alongside pre-trained weights. These models can be used for prediction, feature extraction, and fine-tuning.

Weights are downloaded automatically when instantiating a model.

## (a) The Ising Model – try your show that the square lattice data can be trained perfectly using the embeddings of Inception.

Get the embeddings first, then build a classifier

Solution to (a):

In [10]:
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt

import jax.numpy as jnp
from jax import jit, vmap

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as keras

from sklearn.model_selection import train_test_split

#import keras
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import optimizers

Using TensorFlow backend.


In [11]:
tf.compat.v1.enable_eager_execution()

In [12]:
# Import the data and shape it for training
with tf.device('/CPU:0'):

    N = 250
    nx, ny = 32, 32

    Xsq = np.ndarray((4*N,nx,ny,1))
    ysq = np.ndarray(4*N)

    for i in np.arange(N):
        Xsq[i + 0*N] = np.loadtxt("./square_T1/square_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
        ysq[i + 0*N] = 0
        Xsq[i + 1*N] = np.loadtxt("./square_T2/square_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
        ysq[i + 1*N] = 1
        Xsq[i + 2*N] = np.loadtxt("./square_T3/square_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
        ysq[i + 2*N] = 2
        Xsq[i + 3*N] = np.loadtxt("./square_T4/square_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
        ysq[i + 3*N] = 3

    Xsq_train, Xsq_test, ysq_train, ysq_test = train_test_split(Xsq, ysq, test_size=0.2, random_state=0)
    Xsq_train, Xsq_test = np.repeat(Xsq_train, 8, 1), np.repeat(Xsq_test, 8, 1)
    Xsq_train, Xsq_test = np.repeat(Xsq_train, 8, 2), np.repeat(Xsq_test, 8, 2)
    Xsq_train, Xsq_test = np.repeat(Xsq_train, 3, 3), np.repeat(Xsq_test, 3, 3)
    Xsq_train, Xsq_test = tf.cast(Xsq_train, tf.float32), tf.cast(Xsq_test, tf.float32)

In [69]:
# Get the inception model without last layer and disable training
with tf.device('/CPU:0'):
    base_model = keras.applications.inception_v3.InceptionV3(include_top=False, 
                                                            weights='imagenet', 
                                                            input_shape=Xsq_train[0].shape
                                                            )

    base_model.trainable = False

In [70]:
# Get the embedded data
global_avg_layer = keras.layers.GlobalAveragePooling2D()
Xsq_train_emb, Xsq_test_emb = global_avg_layer(base_model.predict(Xsq_train)), global_avg_layer(base_model.predict(Xsq_test))
#test = base_model(Xsq_train)

In [71]:
Xsq_train_emb

<tf.Tensor: shape=(800, 2048), dtype=float32, numpy=
array([[6.8060863e-01, 7.3173773e-03, 0.0000000e+00, ..., 0.0000000e+00,
        2.5865093e-01, 2.0735666e-01],
       [3.8926208e-01, 1.3012396e-01, 9.8542706e-04, ..., 1.2677726e+00,
        2.5907698e-01, 2.1452351e-02],
       [1.8170083e-01, 1.6207559e-02, 7.5311237e-04, ..., 1.4652914e+00,
        3.1413925e-01, 5.6133777e-01],
       ...,
       [2.0973858e-01, 0.0000000e+00, 0.0000000e+00, ..., 1.1741548e+00,
        3.6734562e-02, 2.3487984e-01],
       [7.4135566e-01, 0.0000000e+00, 0.0000000e+00, ..., 6.8261755e-01,
        5.0442830e-02, 1.0479007e-02],
       [8.2162106e-01, 0.0000000e+00, 0.0000000e+00, ..., 8.7034525e-03,
        6.5520979e-02, 5.3103969e-02]], dtype=float32)>

In [72]:
# According to documentation, this should work for tensorflow version 2.x
Xsq_train_emb.numpy()

array([[6.8060863e-01, 7.3173773e-03, 0.0000000e+00, ..., 0.0000000e+00,
        2.5865093e-01, 2.0735666e-01],
       [3.8926208e-01, 1.3012396e-01, 9.8542706e-04, ..., 1.2677726e+00,
        2.5907698e-01, 2.1452351e-02],
       [1.8170083e-01, 1.6207559e-02, 7.5311237e-04, ..., 1.4652914e+00,
        3.1413925e-01, 5.6133777e-01],
       ...,
       [2.0973858e-01, 0.0000000e+00, 0.0000000e+00, ..., 1.1741548e+00,
        3.6734562e-02, 2.3487984e-01],
       [7.4135566e-01, 0.0000000e+00, 0.0000000e+00, ..., 6.8261755e-01,
        5.0442830e-02, 1.0479007e-02],
       [8.2162106e-01, 0.0000000e+00, 0.0000000e+00, ..., 8.7034525e-03,
        6.5520979e-02, 5.3103969e-02]], dtype=float32)

In [73]:
class classify_FNN:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(num_classes, channels_first=False):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(tf.keras.layers.Dense(32, activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
        
        return model

In [74]:
model = classify_FNN.build(num_classes=4)

In [75]:
model = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(2048, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])


In [76]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [77]:
# fix the labels
y_train = keras.utils.to_categorical(ysq_train, 4)
y_test = keras.utils.to_categorical(ysq_test, 4)

In [78]:
H = model.fit(Xsq_train_emb.numpy(), y_train, validation_data=(Xsq_test_emb.numpy(), y_test), epochs=50,
                  batch_size=64)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 0s 430us/sample - loss: 1.0867 - accuracy: 0.6463 - val_loss: 0.9956 - val_accuracy: 0.7250
Epoch 2/50
800/800 [==============================] - 0s 43us/sample - loss: 0.8880 - accuracy: 0.8737 - val_loss: 0.8387 - val_accuracy: 0.9250
Epoch 3/50
800/800 [==============================] - 0s 43us/sample - loss: 0.8386 - accuracy: 0.9200 - val_loss: 0.8225 - val_accuracy: 0.9400
Epoch 4/50
800/800 [==============================] - 0s 42us/sample - loss: 0.8297 - accuracy: 0.9237 - val_loss: 0.8148 - val_accuracy: 0.9350
Epoch 5/50
800/800 [==============================] - 0s 45us/sample - loss: 0.8066 - accuracy: 0.9475 - val_loss: 0.8411 - val_accuracy: 0.9200
Epoch 6/50
800/800 [==============================] - 0s 48us/sample - loss: 0.8100 - accuracy: 0.9450 - val_loss: 0.8656 - val_accuracy: 0.8600
Epoch 7/50
800/800 [==============================] - 0s 47us/sample - loss: 0.7968

## (b)  [Rayleigh-Bénard Convection](https://en.wikipedia.org/wiki/Rayleigh%E2%80%93B%C3%A9nard_convection)

RB convection, in which a flow is heated from below and cooled  from  top,  is  one  of  the  paradigmatic  system  in  fluid  dynamics. When the temperature difference between the two plates (in dimensionless form Rayleigh number Ra) is beyond certain threshold, hot fluid tends to go up and cold fluid tends to go down, thus forming convection cells. What we supply here are the temperature snapshots from four different Ra, i.e., $Ra=10^{14}$ as `class0`,$Ra= 10^{13}$ as `class1`, $Ra= 10^{12}$ as `class2`,and $Ra= 10^{11}$ as `class3`.  The flow you see is highly turbulent; not only there are big convection cells but also lots of small vortices.  The original dataset  is  around  4000*2000.   We  have  already  downsampled  the  data into the zip file `fluid_org.zip`.

### (1) Train the data in `fluid_org.zip` with inception.  Show that these images can be classified  into  different $Ra$ nicely  with  inception.  

Take the length 2048 embeddings from the Inception model first. Then visualizing how the embeddings distribute using a two component PCA or two component T-SNE, whichever you prefer. Then use any of the previously learned method to train a classifier using the embeddings as input. **Note that T-SNE normally gives you better separation**

In [ ]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
from PIL import Image
imgs = []
labels = []
for file in os.listdir('./fluid_org/'):
    imgs.append(np.array(Image.open('./fluid_org/'+file))/255)
    labels.append(int(file[-5]))

In [ ]:
imgs = np.array(imgs)
labels = np.array(labels)

Solution to (1):

### (2) For advanced use of trainsfer learning from the pre-trained models such as fine-tuning, we need to do the transfer learning in-place, by building a network consists of the Inception and your classifier layers. 
Freeze the part you take from Inception, train
the model and report the accuracy. Then do the fine-tuning. Report
how much increase of accuracy you can manage to get. Fine tuning
by making the top few layer of the Inception model trainable instead
of freezing all the layers. Due to the slowness of training, unleash the
layers one by one. Make comments about how the accuracy change. It is
highly recommended that you train this on Google Colab with the GPU
activated.

Solution to (2):

### (3) Explore the potential of transfer learning on cropped data `fluid-crop`, which are randomly choosen regions of 100*100 pixels from each original 4000*2000 pictures, i.e.,just around 1% of the original picture! 
You can use either method you use in (1) or (2).

Solution to (3):

### (4) Build your own classifier for (2) and (3) without using Inception. Compare the performance of your own classifier with the result in (2) and (3)

Solution to (4):

### (5) Continue (3), construct two examples where a different layer's output is used as the embedding. There are over 300 layers in Inception. Pick one at around the 100th layer and one at around 200th layer. The exact layer you pick is based on your preference. Show the following.
- (i) The distributions of the embeddings similar to what you've done in (1). Together with the result you get in (1), comment the similarity and difference between what you get using the three embedding layers.
- (ii) What is the test accuracy of the three classifiers. What is the test accuracy of the three classifiers? For speeding up the training you can choose to get the embeddings first and put those into a classifier, as you did in (1).

Solution to (5):